# YOLO11 Hyperparameter Tuning

**Enhanced with Automated Hyperparameter Optimization**

Based on:
- [Ultralytics Hyperparameter Tuning Guide](https://docs.ultralytics.com/guides/hyperparameter-tuning/)
- [Ray Tune Integration](https://docs.ultralytics.com/integrations/ray-tune/)

## Introduction

This notebook demonstrates automated hyperparameter tuning for YOLO11 models using:
1. **Built-in Genetic Algorithm Tuner** - Simple, no extra dependencies
2. **Ray Tune Integration** - Advanced optimization with parallel execution

### What Gets Tuned?

The tuner optimizes:
- **Learning rate**: `lr0`, `lrf`
- **Optimizer**: `momentum`, `weight_decay`
- **Loss weights**: `box`, `cls`, `dfl`
- **Augmentation**: HSV, rotation, translation, scale, shear, perspective, flip, mosaic, mixup

---

## 1. Setup and Imports

In [ ]:
import os
import sys
import shutil
import random
import yaml
from pathlib import Path
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd

# Check if we're in a notebook subdirectory
if Path.cwd().name == 'notebooks':
    os.chdir('..')

print(f"Working directory: {os.getcwd()}")
print(f"Python version: {sys.version}")

## 2. Install Dependencies

Install Ultralytics and optionally Ray Tune for advanced optimization.

In [ ]:
# Basic installation
!pip install -q ultralytics

# Optional: Install Ray Tune for advanced optimization
# Uncomment the line below to enable Ray Tune
# !pip install -q "ray[tune]" wandb

from ultralytics import YOLO
import torch

print(f"\n{'='*60}")
print("Installation Complete!")
print(f"{'='*60}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"{'='*60}\n")

## 3. Dataset Preparation

Reuse the dataset preparation from the original notebook.

In [ ]:
def train_val_split(datapath, train_pct=0.9):
    """
    Split dataset into train and validation folders.
    
    Args:
        datapath: Path to custom_data folder containing images/ and labels/
        train_pct: Percentage of data to use for training (default 0.9)
    """
    datapath = Path(datapath)
    images_dir = datapath / 'images'
    labels_dir = datapath / 'labels'
    
    if not images_dir.exists():
        print(f"❌ Error: {images_dir} does not exist!")
        return
    
    # Create output directories
    data_root = Path('data')
    for split in ['train', 'validation']:
        (data_root / split / 'images').mkdir(parents=True, exist_ok=True)
        (data_root / split / 'labels').mkdir(parents=True, exist_ok=True)
    
    # Get all image files
    image_files = list(images_dir.glob('*.jpg')) + list(images_dir.glob('*.png')) + list(images_dir.glob('*.jpeg'))
    print(f"Found {len(image_files)} images")
    
    # Shuffle and split
    random.shuffle(image_files)
    split_idx = int(len(image_files) * train_pct)
    train_files = image_files[:split_idx]
    val_files = image_files[split_idx:]
    
    print(f"Train: {len(train_files)} images")
    print(f"Validation: {len(val_files)} images")
    
    # Copy files
    for file_list, split in [(train_files, 'train'), (val_files, 'validation')]:
        for img_file in file_list:
            # Copy image
            shutil.copy2(img_file, data_root / split / 'images' / img_file.name)
            
            # Copy corresponding label
            label_file = labels_dir / f"{img_file.stem}.txt"
            if label_file.exists():
                shutil.copy2(label_file, data_root / split / 'labels' / label_file.name)
    
    print("\n✅ Dataset split complete!")
    print(f"Output directory: {data_root.absolute()}")

# Only run if dataset hasn't been split yet
if not Path('data/train').exists():
    train_val_split('custom_data', train_pct=0.9)
else:
    print("✅ Dataset already split!")

In [ ]:
def create_data_yaml(path_to_classes_txt, path_to_data_yaml):
    """
    Create YOLO data.yaml configuration file.
    """
    if not os.path.exists(path_to_classes_txt):
        print(f'❌ classes.txt not found at {path_to_classes_txt}')
        return None
    
    with open(path_to_classes_txt, 'r') as f:
        classes = [line.strip() for line in f.readlines() if line.strip()]
    
    num_classes = len(classes)
    
    # Create data dictionary
    data = {
        'path': str(Path('data').absolute()),
        'train': 'train/images',
        'val': 'validation/images',
        'nc': num_classes,
        'names': classes
    }
    
    # Write YAML file
    with open(path_to_data_yaml, 'w') as f:
        yaml.dump(data, f, sort_keys=False)
    
    print(f'✅ Created config file at {path_to_data_yaml}')
    print(f'Number of classes: {num_classes}')
    print(f'Classes: {classes}')
    
    return data

# Create YAML config
classes_txt_path = 'custom_data/classes.txt'
data_yaml_path = 'data.yaml'

if not Path(data_yaml_path).exists():
    config = create_data_yaml(classes_txt_path, data_yaml_path)
else:
    print(f"✅ {data_yaml_path} already exists!")
    with open(data_yaml_path, 'r') as f:
        config = yaml.safe_load(f)
    print(f"Classes: {config['names']}")

## 4. Hyperparameter Tuning Configuration

### Choose Your Tuning Method

**Method 1: Built-in Genetic Algorithm** (Recommended for beginners)
- No extra dependencies
- Uses mutation and crossover
- Simple to use
- Good for 30-300 iterations

**Method 2: Ray Tune** (Advanced users)
- Requires `ray[tune]` installation
- Bayesian optimization, Hyperband
- Parallel execution
- Better for large search spaces

In [ ]:
# ============================================================
# HYPERPARAMETER TUNING CONFIGURATION
# ============================================================

# Choose tuning method: 'genetic' or 'ray_tune'
TUNING_METHOD = 'genetic'  # Change to 'ray_tune' for Ray Tune

# Base configuration
BASE_CONFIG = {
    'data': 'data.yaml',
    'model': 'yolo11s.pt',    # Start with small model for faster tuning
    'epochs': 30,              # Epochs per iteration
    'imgsz': 640,
    'batch': -1,               # Auto batch size
    'project': 'runs/tune',
    'plots': False,            # Disable plots during tuning to save time
    'save': False,             # Only save best model
    'val': True,               # Validate after each iteration
}

# Device configuration
if torch.cuda.is_available():
    BASE_CONFIG['device'] = 0
elif torch.backends.mps.is_available():
    BASE_CONFIG['device'] = 'mps'
else:
    BASE_CONFIG['device'] = 'cpu'

# Genetic Algorithm Tuning Settings
GENETIC_CONFIG = {
    'iterations': 30,          # Number of tuning iterations (10-300 recommended)
    'optimizer': 'AdamW',      # Optimizer to use
}

# Ray Tune Settings (only used if TUNING_METHOD='ray_tune')
RAY_TUNE_CONFIG = {
    'use_ray': True,
    'iterations': 20,          # Number of trials
    'grace_period': 10,        # Min epochs before early stopping
    'gpu_per_trial': 1 if torch.cuda.is_available() else 0,
}

# Custom search space (optional)
# Uncomment and modify to customize hyperparameter ranges
CUSTOM_SEARCH_SPACE = {
    # Learning rate
    # 'lr0': (1e-5, 1e-1),       # Initial learning rate
    # 'lrf': (0.01, 1.0),        # Final learning rate fraction
    
    # Optimizer
    # 'momentum': (0.6, 0.98),   # SGD momentum
    # 'weight_decay': (0.0, 0.001),  # Optimizer weight decay
    
    # Augmentation
    # 'hsv_h': (0.0, 0.1),       # HSV-Hue augmentation
    # 'hsv_s': (0.0, 0.9),       # HSV-Saturation augmentation
    # 'hsv_v': (0.0, 0.9),       # HSV-Value augmentation
    # 'degrees': (0.0, 45.0),    # Rotation degrees
    # 'translate': (0.0, 0.9),   # Translation fraction
    # 'scale': (0.0, 0.9),       # Scaling factor
    # 'shear': (0.0, 10.0),      # Shear degrees
    # 'perspective': (0.0, 0.001),  # Perspective transform
    # 'flipud': (0.0, 1.0),      # Vertical flip probability
    # 'fliplr': (0.0, 1.0),      # Horizontal flip probability
    # 'mosaic': (0.0, 1.0),      # Mosaic augmentation probability
    # 'mixup': (0.0, 1.0),       # Mixup augmentation probability
}

print("Hyperparameter Tuning Configuration")
print("=" * 60)
print(f"Method: {TUNING_METHOD.upper()}")
print(f"Model: {BASE_CONFIG['model']}")
print(f"Epochs per iteration: {BASE_CONFIG['epochs']}")
print(f"Device: {BASE_CONFIG['device']}")
if TUNING_METHOD == 'genetic':
    print(f"Iterations: {GENETIC_CONFIG['iterations']}")
    print(f"Total training runs: {GENETIC_CONFIG['iterations']}")
else:
    print(f"Trials: {RAY_TUNE_CONFIG['iterations']}")
print(f"Custom search space: {'Yes' if CUSTOM_SEARCH_SPACE else 'No (using defaults)'}")
print("=" * 60)

## 5. Run Hyperparameter Tuning

This will:
1. Initialize the model
2. Run multiple training iterations with different hyperparameters
3. Track the best configuration
4. Save results to CSV/logs

**Note:** This can take several hours depending on:
- Number of iterations
- Epochs per iteration
- Dataset size
- Hardware (GPU recommended)

In [ ]:
# Initialize model
model = YOLO(BASE_CONFIG['model'])

print("\n" + "="*60)
print(f"🚀 Starting Hyperparameter Tuning ({TUNING_METHOD.upper()})")
print("="*60)
print("\nThis may take several hours...\n")

# Merge configurations
tune_kwargs = BASE_CONFIG.copy()

if TUNING_METHOD == 'genetic':
    # Built-in genetic algorithm tuning
    tune_kwargs.update(GENETIC_CONFIG)
    if CUSTOM_SEARCH_SPACE:
        tune_kwargs['space'] = CUSTOM_SEARCH_SPACE
    
    # Run tuning
    results = model.tune(**tune_kwargs)
    
elif TUNING_METHOD == 'ray_tune':
    # Ray Tune optimization
    try:
        from ray import tune as ray_tune_module
        
        tune_kwargs.update(RAY_TUNE_CONFIG)
        
        # Convert custom search space to Ray Tune format if provided
        if CUSTOM_SEARCH_SPACE:
            ray_space = {}
            for key, (low, high) in CUSTOM_SEARCH_SPACE.items():
                ray_space[key] = ray_tune_module.uniform(low, high)
            tune_kwargs['space'] = ray_space
        
        # Run Ray Tune
        results = model.tune(**tune_kwargs)
        
    except ImportError:
        print("❌ Ray Tune not installed!")
        print("Run: pip install 'ray[tune]'")
        print("Falling back to genetic algorithm...")
        tune_kwargs.update(GENETIC_CONFIG)
        results = model.tune(**tune_kwargs)

print("\n" + "="*60)
print("✅ Hyperparameter Tuning Complete!")
print("="*60)

## 6. Analyze Tuning Results

Review the evolution of hyperparameters and fitness scores across iterations.

In [ ]:
# Load tuning results
tune_results_csv = Path('runs/tune/tune/evolve.csv')

if tune_results_csv.exists():
    df = pd.read_csv(tune_results_csv)
    
    print("\n" + "="*60)
    print("TUNING RESULTS SUMMARY")
    print("="*60)
    
    # Show best iteration
    best_idx = df['fitness'].idxmax()
    best_fitness = df.loc[best_idx, 'fitness']
    
    print(f"\nBest iteration: {best_idx + 1}")
    print(f"Best fitness: {best_fitness:.4f}")
    
    print("\nBest hyperparameters:")
    print("-" * 60)
    
    # Display key hyperparameters
    key_params = ['lr0', 'lrf', 'momentum', 'weight_decay', 'box', 'cls', 'dfl',
                  'hsv_h', 'hsv_s', 'hsv_v', 'degrees', 'translate', 'scale',
                  'shear', 'perspective', 'flipud', 'fliplr', 'mosaic', 'mixup']
    
    for param in key_params:
        if param in df.columns:
            value = df.loc[best_idx, param]
            print(f"  {param:15s}: {value:.6f}")
    
    print("\n" + "="*60)
    
    # Show evolution statistics
    print("\nEvolution Statistics:")
    print("-" * 60)
    print(f"Total iterations: {len(df)}")
    print(f"Initial fitness: {df['fitness'].iloc[0]:.4f}")
    print(f"Final fitness: {df['fitness'].iloc[-1]:.4f}")
    print(f"Best fitness: {best_fitness:.4f}")
    print(f"Improvement: {(best_fitness - df['fitness'].iloc[0]) / df['fitness'].iloc[0] * 100:.2f}%")
    
    # Plot fitness evolution
    plt.figure(figsize=(14, 10))
    
    # Subplot 1: Fitness over iterations
    plt.subplot(3, 1, 1)
    plt.plot(df.index + 1, df['fitness'], marker='o', linewidth=2, markersize=4)
    plt.axhline(y=best_fitness, color='r', linestyle='--', label=f'Best: {best_fitness:.4f}')
    plt.xlabel('Iteration')
    plt.ylabel('Fitness')
    plt.title('Fitness Evolution Over Iterations', fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Subplot 2: Key metrics evolution
    plt.subplot(3, 1, 2)
    metrics = ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
    available_metrics = [m for m in metrics if m in df.columns]
    
    for metric in available_metrics:
        plt.plot(df.index + 1, df[metric], marker='o', linewidth=2, markersize=3, label=metric.split('/')[-1])
    
    plt.xlabel('Iteration')
    plt.ylabel('Score')
    plt.title('Metrics Evolution', fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Subplot 3: Learning rate evolution
    plt.subplot(3, 1, 3)
    if 'lr0' in df.columns:
        plt.plot(df.index + 1, df['lr0'], marker='o', linewidth=2, markersize=4, label='lr0', color='orange')
    if 'lrf' in df.columns:
        plt.plot(df.index + 1, df['lrf'], marker='s', linewidth=2, markersize=4, label='lrf', color='green')
    
    plt.xlabel('Iteration')
    plt.ylabel('Value')
    plt.title('Learning Rate Evolution', fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('runs/tune/tune/evolution_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n✅ Evolution plot saved to: runs/tune/tune/evolution_analysis.png")
    
else:
    print("⚠️ Tuning results not found. Make sure tuning completed successfully.")

## 7. Train Final Model with Best Hyperparameters

Now train a full model using the best hyperparameters discovered during tuning.

In [ ]:
# Load best hyperparameters
if tune_results_csv.exists():
    df = pd.read_csv(tune_results_csv)
    best_idx = df['fitness'].idxmax()
    
    # Extract best hyperparameters
    best_params = {}
    for col in df.columns:
        if col not in ['fitness', 'metrics/precision(B)', 'metrics/recall(B)', 
                       'metrics/mAP50(B)', 'metrics/mAP50-95(B)']:
            best_params[col] = df.loc[best_idx, col]
    
    print("\n" + "="*60)
    print("🚀 Training Final Model with Best Hyperparameters")
    print("="*60)
    
    # Final training configuration
    FINAL_CONFIG = {
        'data': 'data.yaml',
        'model': 'yolo11s.pt',  # Can upgrade to 'yolo11m.pt' or 'yolo11l.pt' for better accuracy
        'epochs': 100,           # More epochs for final training
        'imgsz': 640,
        'batch': -1,
        'device': BASE_CONFIG['device'],
        'project': 'runs/detect',
        'name': 'final_tuned',
        'exist_ok': True,
        'patience': 50,
        'save': True,
        'plots': True,
    }
    
    # Merge best hyperparameters
    FINAL_CONFIG.update(best_params)
    
    # Initialize fresh model
    final_model = YOLO(FINAL_CONFIG['model'])
    
    # Train with best hyperparameters
    final_results = final_model.train(**FINAL_CONFIG)
    
    print("\n" + "="*60)
    print("✅ Final Model Training Complete!")
    print("="*60)
    print(f"\nBest weights saved at: runs/detect/final_tuned/weights/best.pt")
    print(f"Training plots saved at: runs/detect/final_tuned/results.png")
    
else:
    print("⚠️ Cannot find best hyperparameters. Run tuning first!")

## 8. Visualize Final Results

In [ ]:
# Display final training results
results_plot = Path('runs/detect/final_tuned/results.png')
if results_plot.exists():
    img = plt.imread(str(results_plot))
    plt.figure(figsize=(16, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Final Training Results (Tuned Model)', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Results plot not found")

# Display confusion matrix
confusion_matrix = Path('runs/detect/final_tuned/confusion_matrix.png')
if confusion_matrix.exists():
    img = plt.imread(str(confusion_matrix))
    plt.figure(figsize=(12, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Confusion Matrix (Tuned Model)', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Confusion matrix not found")

## 9. Validate Tuned Model

In [ ]:
# Load best tuned model
best_model_path = 'runs/detect/final_tuned/weights/best.pt'

if Path(best_model_path).exists():
    tuned_model = YOLO(best_model_path)
    
    print("Running validation on tuned model...\n")
    
    # Validate
    metrics = tuned_model.val(data='data.yaml')
    
    print("\n" + "="*60)
    print("TUNED MODEL VALIDATION METRICS")
    print("="*60)
    print(f"mAP50:     {metrics.box.map50:.4f} ({metrics.box.map50*100:.2f}%)")
    print(f"mAP50-95:  {metrics.box.map:.4f} ({metrics.box.map*100:.2f}%)")
    print(f"Precision: {metrics.box.mp:.4f}")
    print(f"Recall:    {metrics.box.mr:.4f}")
    print("="*60)
else:
    print(f"⚠️ Model not found at {best_model_path}")

## 10. Compare: Baseline vs Tuned Model

Compare performance between a baseline model and your tuned model.

In [ ]:
# Load baseline model metrics if available
baseline_results = Path('runs/detect/train/results.csv')
tuned_results = Path('runs/detect/final_tuned/results.csv')

if baseline_results.exists() and tuned_results.exists():
    baseline_df = pd.read_csv(baseline_results)
    tuned_df = pd.read_csv(tuned_results)
    
    # Strip whitespace from column names
    baseline_df.columns = baseline_df.columns.str.strip()
    tuned_df.columns = tuned_df.columns.str.strip()
    
    # Get final metrics
    metrics_to_compare = ['metrics/precision(B)', 'metrics/recall(B)', 
                          'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
    
    print("\n" + "="*60)
    print("BASELINE vs TUNED MODEL COMPARISON")
    print("="*60)
    
    comparison_data = []
    for metric in metrics_to_compare:
        if metric in baseline_df.columns and metric in tuned_df.columns:
            baseline_val = baseline_df[metric].iloc[-1]
            tuned_val = tuned_df[metric].iloc[-1]
            improvement = ((tuned_val - baseline_val) / baseline_val * 100)
            
            metric_name = metric.split('/')[-1].replace('(B)', '')
            print(f"\n{metric_name}:")
            print(f"  Baseline: {baseline_val:.4f}")
            print(f"  Tuned:    {tuned_val:.4f}")
            print(f"  Change:   {improvement:+.2f}%")
            
            comparison_data.append({
                'Metric': metric_name,
                'Baseline': baseline_val,
                'Tuned': tuned_val,
                'Improvement': improvement
            })
    
    print("\n" + "="*60)
    
    # Visualize comparison
    if comparison_data:
        comp_df = pd.DataFrame(comparison_data)
        
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        # Bar chart comparison
        x = np.arange(len(comp_df))
        width = 0.35
        
        axes[0].bar(x - width/2, comp_df['Baseline'], width, label='Baseline', alpha=0.8)
        axes[0].bar(x + width/2, comp_df['Tuned'], width, label='Tuned', alpha=0.8)
        axes[0].set_xlabel('Metric')
        axes[0].set_ylabel('Score')
        axes[0].set_title('Baseline vs Tuned Model Performance', fontweight='bold')
        axes[0].set_xticks(x)
        axes[0].set_xticklabels(comp_df['Metric'], rotation=45, ha='right')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # Improvement chart
        colors = ['green' if x > 0 else 'red' for x in comp_df['Improvement']]
        axes[1].barh(comp_df['Metric'], comp_df['Improvement'], color=colors, alpha=0.8)
        axes[1].set_xlabel('Improvement (%)')
        axes[1].set_title('Performance Improvement', fontweight='bold')
        axes[1].axvline(x=0, color='black', linestyle='-', linewidth=0.8)
        axes[1].grid(True, alpha=0.3, axis='x')
        
        plt.tight_layout()
        plt.savefig('runs/tune/baseline_vs_tuned_comparison.png', dpi=150, bbox_inches='tight')
        plt.show()
        
        print("\n✅ Comparison plot saved to: runs/tune/baseline_vs_tuned_comparison.png")
        
else:
    print("⚠️ Baseline or tuned results not found for comparison.")
    print("Run the original training notebook first to create a baseline.")

## 11. Export Tuned Model

In [ ]:
if Path(best_model_path).exists():
    tuned_model = YOLO(best_model_path)
    
    # Export to ONNX format
    print("Exporting tuned model to ONNX format...")
    onnx_path = tuned_model.export(format='onnx', dynamic=True)
    print(f"✅ ONNX model exported to: {onnx_path}")
    
    # Uncomment to export to other formats:
    # coreml_path = tuned_model.export(format='coreml')  # For iOS/macOS
    # tflite_path = tuned_model.export(format='tflite')  # For mobile/edge devices
else:
    print(f"⚠️ Model not found at {best_model_path}")

## 12. Summary and Recommendations

### What We Did

1. **Automated Hyperparameter Search**: Used genetic algorithm or Ray Tune to find optimal hyperparameters
2. **Evolution Tracking**: Monitored fitness and metrics across iterations
3. **Final Training**: Trained a full model with best hyperparameters
4. **Performance Comparison**: Compared baseline vs tuned model

### Key Files

- **Tuning results**: `runs/tune/tune/evolve.csv`
- **Best tuned model**: `runs/detect/final_tuned/weights/best.pt`
- **Training plots**: `runs/detect/final_tuned/results.png`
- **ONNX export**: `runs/detect/final_tuned/weights/best.onnx`

### Next Steps

1. **Further Tuning**:
   - Increase iterations (100-300) for better results
   - Try Ray Tune with Bayesian optimization
   - Customize search space for specific parameters

2. **Model Improvements**:
   - Use larger model (`yolo11m.pt`, `yolo11l.pt`) for higher accuracy
   - Increase final training epochs (150-200)
   - Add more training data if possible

3. **Advanced Techniques**:
   - Enable MongoDB for distributed tuning across machines
   - Use ensemble methods with multiple tuned models
   - Fine-tune with transfer learning from domain-specific weights

### Resources

- [Ultralytics Hyperparameter Tuning Guide](https://docs.ultralytics.com/guides/hyperparameter-tuning/)
- [Ray Tune Integration](https://docs.ultralytics.com/integrations/ray-tune/)
- [Ultralytics Documentation](https://docs.ultralytics.com/)

---

**Happy tuning!** 🎯🔧